In [1]:
from __future__ import print_function
import os
import numpy as np
np.random.seed(1337)

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils.np_utils import to_categorical
from keras.layers import Dense, Input, Flatten
from keras.layers import Conv1D, MaxPooling1D, Embedding
from keras.models import Model
import sys

Using Theano backend.
Using gpu device 0: GeForce GT 730M (CNMeM is disabled, cuDNN 5103)


### Importing texts

In [2]:
import xml.etree.ElementTree as ET
print('Processing text dataset')

tree = ET.parse("/home/jeet/Academics/CS671/Project/Restaurants_Train.xml")
corpus = tree.getroot()
sentences = [] # List of list of sentences.
sent = corpus.findall('.//sentence')
for s in sent:
    sentences.append(s.find('text').text)

print ('Generated list of sentences..')

Processing text dataset
Generated list of sentences..


In [38]:
MAX_SEQ_LENGTH = 69
MAX_NB_WORDS = 40000
EMBEDDING_DIM = 300

#### Indexing Word Vectors

In [4]:
print('Indexing word vectors.')

embeddings_index = {}
f = open('glove.6B/glove.6B.300d.txt')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

print('Found %s word vectors.' % len(embeddings_index))

Indexing word vectors.
Found 400000 word vectors.


#### vectorize the text samples into a 2D integer tensor and padding the sentences

In [5]:
tokenizer = Tokenizer(nb_words=MAX_NB_WORDS, lower=False)
tokenizer.fit_on_texts(sentences)
sequences = tokenizer.texts_to_sequences(sentences)

word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))
print ("Let's have a quick look at the word_index data..")
print (list(word_index.items())[:10])
# print (word_index['limited'])

Found 5250 unique tokens.
Let's have a quick look at the word_index data..
[('rasamalai', 2402), ('limited', 647), ('arrives', 4234), ('legacies', 2403), ('raining', 2404), ('saves', 1617), ('AN', 2405), ('meatsauce', 4956), ('sleek', 1237), ('four', 648)]


In [6]:
data = pad_sequences(sequences, maxlen=MAX_SEQ_LENGTH, padding='post')
print('Shape of data tensor:', data.shape)

Shape of data tensor: (3044, 69)


#### defining output data

In [35]:
import nltk
from keras.preprocessing.text import text_to_word_sequence
raw_output = corpus.findall('.//sentence')
train_out= np.zeros(shape=(3044,69))
i=0
for output in raw_output:
    s = text_to_word_sequence(output.find('text').text, lower=False)
    indices = np.zeros(MAX_SEQ_LENGTH)
    
    aspectTerms = output.find('aspectTerms')
    if (aspectTerms):
        aspectTerm = aspectTerms.findall('aspectTerm')
        if (aspectTerm):
            for aspect_term in aspectTerm:
                try:
                    indices[s.index(aspect_term.attrib['term'])] = 1
#                     print (indices)
                except:
                    continue
    train_out[i] = indices
    i=i+1

/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:11: FutureWarning: The behavior of this method will change in future versions.  Use specific 'len(elem)' or 'elem is not None' test instead.


In [37]:
print (train_out.shape)

(3044, 69)


#### Preparing Embedding Layer

In [40]:
print('Preparing embedding matrix.')

# prepare embedding matrix
nb_words = len(word_index)
embedding_matrix = np.zeros((nb_words + 1, 300))
for word, i in word_index.items():
    if i > MAX_NB_WORDS:
        continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector

# load pre-trained word embeddings into an Embedding layer
# note that we set trainable = False so as to keep the embeddings fixed
embedding_layer = Embedding(nb_words + 1,
                            300,
                            weights=[embedding_matrix],
                            input_length=MAX_SEQ_LENGTH,
                            trainable=False)

Preparing embedding matrix.


### Defining and Training Model

In [41]:
from keras.models import Sequential
from keras.layers.convolutional import Convolution1D, Convolution2D
from keras.layers.convolutional import MaxPooling1D, MaxPooling2D
from keras.layers.core import Activation
from keras.layers.core import Flatten
from keras.layers.core import Dense
from keras.preprocessing import sequence
from keras.optimizers import *
from keras.regularizers import l2

In [47]:
print('Training model.')

model = Sequential()
model.add(embedding_layer)
model.add(Convolution1D(100, 5, border_mode="same", input_shape=(65, 300)))
model.add(Activation("tanh"))
model.add(MaxPooling1D(pool_length=5))
model.add(Convolution1D(50, 3, border_mode="same"))
model.add(Activation("tanh"))
model.add(MaxPooling1D(pool_length=2))
model.add(Flatten())
model.add(Dense(500))
model.add(Activation("tanh"))
# softmax classifier
model.add(Dense(69, W_regularizer=l2(0.01)))
model.add(Activation("softmax"))



# # train a 1D convnet with global maxpooling
# sequence_input = Input(shape=(MAX_SEQ_LENGTH,), dtype='int32')
# embedded_sequences = embedding_layer(sequence_input)
# x = Conv1D(100, 5, activation='tanh')(embedded_sequences)
# x = MaxPooling1D(5)(x)
# x = Conv1D(50, 5, activation='relu')(x)
# x = MaxPooling1D(5)(x)
# x = Conv1D(128, 5, activation='relu')(x)
# x = MaxPooling1D(35)(x)
# x = Flatten()(x)
# x = Dense(128, activation='relu')(x)
# preds = Dense(len(labels_index), activation='softmax')(x)

# model = Model(sequence_input, preds)
model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['acc'])

Training model.


In [48]:
print (model.summary())

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
embedding_1 (Embedding)          (None, 69, 300)       0           input_1[0][0]                    
____________________________________________________________________________________________________
convolution1d_8 (Convolution1D)  (None, 69, 100)       150100      embedding_1[0][0]                
____________________________________________________________________________________________________
activation_9 (Activation)        (None, 69, 100)       0           convolution1d_8[0][0]            
____________________________________________________________________________________________________
maxpooling1d_8 (MaxPooling1D)    (None, 13, 100)       0           activation_9[0][0]               
___________________________________________________________________________________________

In [50]:
model.fit(data, train_out,
          validation_split=0.1,
          batch_size=10,
          nb_epoch=50
         )

Train on 2739 samples, validate on 305 samples
Epoch 1/50
2739/2739 [==============================] - 6s - loss: 1.0084 - acc: 0.5111 - val_loss: 1.7087 - val_acc: 0.4393
Epoch 2/50
2739/2739 [==============================] - 5s - loss: 0.9730 - acc: 0.5144 - val_loss: 1.7393 - val_acc: 0.4098
Epoch 3/50
2739/2739 [==============================] - 5s - loss: 0.9644 - acc: 0.5210 - val_loss: 1.7671 - val_acc: 0.5115
Epoch 4/50
2739/2739 [==============================] - 5s - loss: 0.9436 - acc: 0.5206 - val_loss: 1.7462 - val_acc: 0.4525
Epoch 5/50
2739/2739 [==============================] - 5s - loss: 0.9266 - acc: 0.5133 - val_loss: 1.7611 - val_acc: 0.4426
Epoch 6/50
2739/2739 [==============================] - 6s - loss: 0.9157 - acc: 0.5225 - val_loss: 1.6720 - val_acc: 0.4656
Epoch 7/50
2739/2739 [==============================] - 5s - loss: 0.8974 - acc: 0.5086 - val_loss: 1.7043 - val_acc: 0.4492
Epoch 8/50
2739/2739 [==============================] - 5s - loss: 0.8983 - ac